<a href="https://colab.research.google.com/github/monya-9/deep-learning-practice/blob/main/02_cnn_cifar10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CNN – CIFAR-10 분류기

CIFAR-10 데이터셋(32×32 컬러 이미지, 10 클래스)을 CNN으로 분류하는 실습입니다.

In [3]:
# 1. 라이브러리 불러오기
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms

In [4]:
# 2. 데이터셋 로드 & 전처리
# CIFAR-10: 32x32 컬러 이미지, 클래스 10개
transform = transforms.Compose([
    transforms.ToTensor(),  # 이미지를 Tensor로 변환
    transforms.Normalize((0.5, 0.5, 0.5),  # RGB 채널 평균
                         (0.5, 0.5, 0.5))  # RGB 채널 표준편차
])

# 학습 데이터
trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                         download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)

# 테스트 데이터
testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                        download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False)

# 클래스 이름
classes = ('plane', 'car', 'bird', 'cat', 'deer',
           'dog', 'frog', 'horse', 'ship', 'truck')

100%|██████████| 170M/170M [00:04<00:00, 41.0MB/s]


1. Normalize로 학습 안정화를 위한 정규화
2. batch_size = 64 설정

In [5]:
# 3. CNN 모델 정의
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, 3, padding=1) # 32x32 → 32x32
        self.pool = nn.MaxPool2d(2, 2)              # 32x32 → 16x16
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)# 16x16 → 16x16
        self.fc1 = nn.Linear(64 * 8 * 8, 256)
        self.fc2 = nn.Linear(256, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 64 * 8 * 8)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
net = SimpleCNN().to(device)

1. view로 feature map을 fc 레이어 입력 형태로 변환
2. ReLU 빈선형 활성화 함수 사용

In [6]:
# 4. 손실함수 & 옵티마이저
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.001)

1. CrossEntropyLoss 다중 클래스 분류에 적합한 손실 함수
2. Adam optimizer 작고 안정적인 학습률을 위해 선택

In [7]:
# 5. 학습
for epoch in range(5):  # 5 epoch
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data[0].to(device), data[1].to(device)

        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
    print(f"[Epoch {epoch+1}] loss: {running_loss/len(trainloader):.3f}")

print('Finished Training')

[Epoch 1] loss: 1.323
[Epoch 2] loss: 0.936
[Epoch 3] loss: 0.768
[Epoch 4] loss: 0.636
[Epoch 5] loss: 0.522
Finished Training


1. 학습 진행 상황을 Epoch 단위로 로그 출력

In [8]:
# 6. 정확도 테스트
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data[0].to(device), data[1].to(device)
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Accuracy on test images: {100 * correct / total:.2f}%")

Accuracy on test images: 72.15%


1. torch.no_grad()로 평가 시 불필요한 gradient 계산을 방지
2. 최종 테스트 정확도를 %로 출력

## 🔹 정확도를 80% 이상으로 높이는 방법
- CNN 층 추가: Conv 레이어를 더 쌓거나 채널 수 늘리기
- 데이터 증강(Data Augmentation): RandomCrop, RandomHorizontalFlip 등
- Dropout 적용: FC 층 전에 Dropout 추가 → 과적합 방지
- 학습 Epoch 늘리기 / 학습률 조정

In [ ]:
# =======================
# 1) 라이브러리 불러오기
# =======================
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

# =======================
# 2) 데이터셋 로드 & 전처리
# =======================
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),  # ⬆ 추가: 데이터 증강
    transforms.RandomCrop(32, padding=4),  # ⬆ 추가: 데이터 증강
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))
])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True,
                                       transform=transforms.Compose([
                                           transforms.ToTensor(),
                                           transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))
                                       ]))
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False)

classes = ('plane', 'car', 'bird', 'cat', 'deer',
           'dog', 'frog', 'horse', 'ship', 'truck')

# =======================
# 3) CNN 모델 정의
# =======================
class ImprovedCNN(nn.Module):  # ⬆ 변경: 모델 이름 & 구조
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3,32,3,padding=1)
        self.conv2 = nn.Conv2d(32,64,3,padding=1)
        self.conv3 = nn.Conv2d(64,128,3,padding=1)  # ⬆ 추가: Conv3
        self.pool = nn.MaxPool2d(2,2)
        self.dropout = nn.Dropout(0.3)             # ⬆ 추가: Dropout
        self.fc1 = nn.Linear(128*4*4,256)          # ⬆ 변경: FC 입력 크기
        self.fc2 = nn.Linear(256,10)

    def forward(self,x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = self.pool(torch.relu(self.conv3(x)))  # ⬆ 추가: Conv3
        x = x.view(-1,128*4*4)
        x = self.dropout(torch.relu(self.fc1(x)))  # ⬆ 추가: Dropout
        return self.fc2(x)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
net = ImprovedCNN().to(device)

# =======================
# 4) 손실함수 & 옵티마이저
# =======================
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.001)

# =======================
# 5) 학습
# =======================
for epoch in range(20):  # ⬆ 변경: Epoch 20으로 증가
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data[0].to(device), data[1].to(device)
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"[Epoch {epoch+1}] loss: {running_loss/len(trainloader):.3f}")

print("Finished Training")

# =======================
# 6) 테스트 정확도
# =======================
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data[0].to(device), data[1].to(device)
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Accuracy on test images: {100 * correct / total:.2f}%")


[Epoch 1] loss: 1.615
[Epoch 2] loss: 1.272
[Epoch 3] loss: 1.097
[Epoch 4] loss: 0.988
[Epoch 5] loss: 0.905
[Epoch 6] loss: 0.850
[Epoch 7] loss: 0.809
